---
---
# **STEP 0 - PSEUDOLABELLING NAMED ENTITY RECOGNITION**
---
---

---
### **IMPORT LIBRARIES AND DEPENDANCIES**
---

In [ ]:
import torch
print(torch.cuda.is_available())                     # True
print(torch.version.__version__)                     # Torch version (e.g., 2.6.0)
print(torch.version.cuda)                            # CUDA version
print(torch.cuda.get_device_name(0))                 # Your GPU name


True
2.6.0+cu124
12.4
Tesla T4


In [ ]:
!pip install seqeval

['Collecting seqeval',
 '  Downloading seqeval-1.2.2.tar.gz (43 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/43.6 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b43.6/43.6 kB\x1b \x1b3.5 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h  Preparing metadata (setup.py) ... \x1b[?25l\x1b[?25hdone',
 'Requirement already satisfied: numpy>=1.14.0 in /usr/local/lib/python3.11/dist-packages (from seqeval) (2.0.2)',
 'Requirement already satisfied: scikit-learn>=0.21.3 in /usr/local/lib/python3.11/dist-packages (from seqeval) (1.6.1)',
 'Requirement already satisfied: scipy>=1.6.0 in /usr/local/lib/python3.11/dist-packages (from scikit-learn>=0.21.3->seqeval) (1.15.3)',
 'Requirement already satisfied: joblib>=1.2.0 in /usr/local/lib/python3.11/dist-packages (from scikit-learn>=0.21.3->seqeval) (1.5.1)',
 'Requirement already satisfied: threadpoolctl>=3.1.0 in /usr/local/lib/python3.11/dist-packages (from scik

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
import torch

---
### **LOAD DATASET**
---

In [ ]:
# Load annotated dataset
df = pd.read_csv("/content/TA_MASTER SHEET - even_semi_annotated.csv")

# Only keep relevant columns
df = df[['story_id', 'judul', 'sentence_id', 'word', 'TYPE2']]


---
### **EXPLORATORY DATA ANALYSIS (EDA)**
---

In [ ]:
# --- Count total number of sentences ---

n_sentences = df[['story_id', 'sentence_id']].drop_duplicates().shape[0]

print(f"Total number of sentences: {n_sentences}")


Total number of sentences: 1157


In [ ]:
# --- Mean number of sentences per story ---

sentences_per_story = df[['story_id', 'sentence_id']].drop_duplicates().groupby('story_id').size()
mean_sentences_per_story = sentences_per_story.mean()

print(f"Mean number of sentences per story: {mean_sentences_per_story:.2f}")


Mean number of sentences per story: 115.70


---
### **DATA PREPARATION**
---

In [ ]:
# Convert labels to unique integers
unique_labels = df['TYPE2'].unique().tolist()
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {v: k for k, v in label2id.items()}

# Create list of sentences (grouped by sentence_id)
grouped = df.groupby(['story_id', 'sentence_id'])
sentences = []
labels = []
story_ids = []

for (story_id, _), group in grouped:
    word_list = group['word'].tolist()
    label_list = group['TYPE2'].map(label2id).tolist()
    sentences.append(word_list)
    labels.append(label_list)
    story_ids.append(story_id)


In [ ]:
# Create HuggingFace dataset
dataset = Dataset.from_dict({
    'tokens': sentences,
    'ner_tags': labels,
    'story_id': story_ids
})

# Train/test split
dataset = dataset.train_test_split(test_size=0.2)

# Load tokenizer
model_checkpoint = "cahya/bert-base-indonesian-1.5G"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    all_labels = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        new_labels = []
        current_label = None

        for word_idx in word_ids:
            if word_idx is None:
                new_labels.append(-100)
            else:
                label = labels[word_idx]
                label_name = id2label[label]

                # Start of a new word
                if word_idx != current_label:
                    current_label = word_idx
                    new_labels.append(label)
                # Continuation of the same word
                else:
                    # Convert B-PER to I-PER for subwords
                    if label_name == "B-PER":
                        new_labels.append(label2id["I-PER"])
                    else:
                        new_labels.append(label)

        all_labels.append(new_labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

In [ ]:
# Apply preprocessing
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Load model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label2id), id2label=id2label, label2id=label2id)


Map:   0%|          | 0/925 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/232 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---
### **EVALUATION METRICS**
---

In [ ]:
from seqeval.metrics import accuracy_score, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_labels = []
    pred_labels = []

    for true, pred in zip(labels, predictions):
        true_seq = []
        pred_seq = []
        for t, p in zip(true, pred):
            if t != -100:
                true_seq.append(id2label[t])
                pred_seq.append(id2label[p])
        true_labels.append(true_seq)
        pred_labels.append(pred_seq)

    return {
        "accuracy": accuracy_score(true_labels, pred_labels),
        "f1": f1_score(true_labels, pred_labels)
    }


---
### **ITERATION 0**
---

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

/tmp/ipython-input-13-2653269769.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 5025211176 (5025211176-institut-teknologi-sepuluh-nopember) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.056651,0.978775,0.888276
2,No log,0.056899,0.982871,0.904965
3,No log,0.063810,0.983430,0.912181
4,No log,0.076109,0.982685,0.908029
5,No log,0.078317,0.982871,0.905233
6,No log,0.087232,0.981940,0.907539
7,No log,0.087647,0.983057,0.911681
8,No log,0.088054,0.983616,0.911932
9,0.042200,0.091546,0.982685,0.907042
10,0.042200,0.091388,0.983243,0.912181


TrainOutput(global_step=580, training_loss=0.036594905395959984, metrics={'train_runtime': 435.1079, 'train_samples_per_second': 21.259, 'train_steps_per_second': 1.333, 'total_flos': 514456800745680.0, 'train_loss': 0.036594905395959984, 'epoch': 10.0})

In [ ]:
# Save final model
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")

('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/vocab.txt',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')

In [ ]:
from transformers import TokenClassificationPipeline
from collections import defaultdict

# Setup pipeline
ner_pipe = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # Merge subwords like B-PER + I-PER
    device=0 if torch.cuda.is_available() else -1
)

# Clean token (remove subword markers like "##")
def clean_word(word):
    return word.replace("##", "").strip()

# Final character storage
storywise_characters = defaultdict(set)

# Process each test sentence
for story_id, tokens in zip(dataset["test"]["story_id"], dataset["test"]["tokens"]):
    sentence = " ".join(tokens)
    preds = ner_pipe(sentence)

    entity_buffer = []
    prev_entity_type = None

    for pred in preds:
        label = pred["entity_group"]
        word = clean_word(pred["word"])

        if label.startswith("PER"):
            # Flush previous entity buffer
            if entity_buffer:
                name = tokenizer.convert_tokens_to_string(entity_buffer).strip().title()
                if name:
                    storywise_characters[story_id].add(name)
            entity_buffer = [word]
        else:
            if entity_buffer:
                name = tokenizer.convert_tokens_to_string(entity_buffer).strip().title()
                if name:
                    storywise_characters[story_id].add(name)
                entity_buffer = []

        prev_entity_type = label

    # Final flush
    if entity_buffer:
        name = tokenizer.convert_tokens_to_string(entity_buffer).strip().title()
        if name:
            storywise_characters[story_id].add(name)

# ✅ Print final result
for story, chars in storywise_characters.items():
    print(f"Story ID: {story}\nCharacters: {sorted(chars)}\n")


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Story ID: 35
Characters: ['Adiknya', 'Anakku', 'Ayahnya', 'Banta Seudang', 'Bungsu', 'Gajah Putih', 'Gajah Putihnya', 'Ibunya', 'Imam', 'Isterinya', 'Kakak - Kakaknya', 'Kucing', 'Mak Toyo', 'Nek', 'Orang Tua Banta Seudang', 'Puteri', 'Raja', 'Raja Yang Buta', 'Yang']

Story ID: 7
Characters: ['Abang', 'Abang Beradik', 'Abangnya', 'Adiknya', 'Ahmad', 'Anak Anaknya', 'Anak Muda', 'Anakku', 'Ayahandanya', 'Ayahnya', 'Bang', 'Bu', 'Burung', 'Burung Merbuk', 'Burung Rajawali', 'Datuk Bendahara', 'Gajah', 'Gajah Putih', 'Ibu', 'Ibunya', 'Isterinya', 'Ita', 'Muhammad', 'Orang', 'Orang Orang Di Kampung', 'Pak Ahmad', 'Pembantu', 'Pemuda', 'Pemuda Tampan', 'Pengawal', 'Perdana Menteri', 'Puteri Bungsu', 'Puteri Yang', 'Raja', 'Raja Yang', 'Rakyat', 'Saudara', 'Tuan Puteri', 'Tuan Puteri Bungsu', 'Wak Pawang', 'Wak Pawang Merbuk']

Story ID: 111
Characters: ['Anak', 'Anak - Anak Kampung', 'Anak Raja', 'Anaknya', 'Anjing', 'Ayah', 'Ayahnya', 'Bundanya', 'Istri Raja', 'Istrinya', 'Kucing', 'Raja'

In [ ]:
import pandas as pd

# Assuming this mapping exists from earlier
story_id_to_title = df.groupby('story_id')['judul'].first().to_dict()

# Flatten the results into one row per character
char_data_flat = {
    "story_id": [],
    "judul": [],
    "character": []
}

for story_id, characters in storywise_characters.items():
    for character in sorted(characters):
        char_data_flat["story_id"].append(story_id)
        char_data_flat["judul"].append(story_id_to_title.get(story_id, "Unknown"))
        char_data_flat["character"].append(character)

# Convert to DataFrame
df_flat = pd.DataFrame(char_data_flat)

# Show it
df_flat


,story_id,judul,character
0,35,Banta Seudang,Adiknya
1,35,Banta Seudang,Anakku
2,35,Banta Seudang,Ayahnya
3,35,Banta Seudang,Banta Seudang
4,35,Banta Seudang,Bungsu
...,...,...,...
154,23,Rawang Tengkuluk,Siti
155,23,Rawang Tengkuluk,Tek Kanah
156,23,Rawang Tengkuluk,Tengku
157,23,Rawang Tengkuluk,Tikus


## comparison

In [ ]:
from collections import defaultdict

ground_truth_characters = defaultdict(list)

for story_id, group in df[df['TYPE2'].isin(['B-PER', 'I-PER'])].groupby(['story_id', 'sentence_id']):
    current_entity = []
    prev_type = None

    for _, row in group.iterrows():
        label = row['TYPE2']
        word = row['word']

        if label == 'B-PER':
            if current_entity:
                full_name = " ".join(current_entity).strip().title()
                ground_truth_characters[story_id[0]].append(full_name)
            current_entity = [word]
        elif label == 'I-PER':
            if prev_type in ['B-PER', 'I-PER']:
                current_entity.append(word)
            else:
                if current_entity:
                    full_name = " ".join(current_entity).strip().title()
                    ground_truth_characters[story_id[0]].append(full_name)
                current_entity = []

        prev_type = label

    # Flush last entity
    if current_entity:
        full_name = " ".join(current_entity).strip().title()
        ground_truth_characters[story_id[0]].append(full_name)


In [ ]:
for story_id in sorted(set(list(ground_truth_characters.keys()) + list(storywise_characters.keys()))):
    gt_set = set(ground_truth_characters.get(story_id, []))
    pred_set = set(storywise_characters.get(story_id, []))

    print(f"\n📚 Story ID: {story_id}")
    print(f"🟢 Ground Truth Characters: {sorted(gt_set)}")
    print(f"🔵 Predicted Characters:     {sorted(pred_set)}")

    matched = gt_set & pred_set
    missed = gt_set - pred_set
    extra = pred_set - gt_set

    print(f"✅ Matched: {sorted(matched)}")
    print(f"❌ Missed:  {sorted(missed)}")
    print(f"🌀 Extra:   {sorted(extra)}")



📚 Story ID: 7
🟢 Ground Truth Characters: ['Abang', 'Abangku', 'Abangnya', 'Adik', 'Adiknya', 'Ahmad', 'Anak Dara', 'Anak Laki Laki', 'Anak Muda', 'Anak Perempuan', 'Ayah', 'Ayahanda', 'Ayahandanya', 'Ayahmu', 'Ayahnya', 'Baginda Raja', 'Bang', 'Bu', 'Burung', 'Burung Kesayangannya', 'Burung Merbuk', 'Burung Merbuknya', 'Burung Rajawali', 'Datuk Bendahara', 'Gajah Gajah', 'Gajah Putih', 'Ibu', 'Ibu Ahmad', 'Ibumu', 'Ibunya', 'Isterinya', 'Istri', 'Mak', 'Mak Inang', 'Merbuk', 'Muhammad', 'Orang Muda', 'Orang Tua', 'Pak Ahmad', 'Pawang Merbuk', 'Pembantu', 'Pembantunya', 'Pemuda', 'Pengawal', 'Pengawal Istana', 'Pengawal Raja', 'Perdana Menteri', 'Permaisuri', 'Puteranya', 'Puteri', 'Puteri Bungsu', 'Puteri Sulung', 'Putri Bungsu', 'Raja', 'Rajawali', 'Rakyat', 'Rakyatnya', 'Suami', 'Tuan Puteri', 'Tuan Puteri Bungsu', 'Wak', 'Wak Pawang', 'Wak Pawang Merbuk', 'Yang Maha Kuasa']
🔵 Predicted Characters:     ['Abang', 'Abang Beradik', 'Abangnya', 'Adiknya', 'Ahmad', 'Anak Anaknya', 'Anak 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def precision_recall_f1(y_true, y_pred):
    true_set = set(y_true)
    pred_set = set(y_pred)
    intersection = true_set & pred_set

    precision = len(intersection) / len(pred_set) if pred_set else 0.0
    recall = len(intersection) / len(true_set) if true_set else 0.0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0

    return precision, recall, f1


In [ ]:
eval_data = {
    "story_id": [],
    "judul": [],
    "ground_truth": [],
    "predicted": [],
    "matched": [],
    "precision": [],
    "recall": [],
    "f1_score": []
}

for story_id in sorted(set(ground_truth_characters) | set(storywise_characters)):
    gt = ground_truth_characters.get(story_id, [])
    pred = list(storywise_characters.get(story_id, set()))
    matched = list(set(gt) & set(pred))

    p, r, f1 = precision_recall_f1(gt, pred)

    eval_data["story_id"].append(story_id)
    eval_data["judul"].append(story_id_to_title.get(story_id, "Unknown"))
    eval_data["ground_truth"].append(len(gt))
    eval_data["predicted"].append(len(pred))
    eval_data["matched"].append(len(matched))
    eval_data["precision"].append(round(p, 3))
    eval_data["recall"].append(round(r, 3))
    eval_data["f1_score"].append(round(f1, 3))

df_eval = pd.DataFrame(eval_data)
print(df_eval)


   story_id                          judul  ground_truth  predicted  matched  \
0         7             Tuah Burung Merbak           539         41       30   
1        10  Sipakpak Kunal Dan Nagai Sori           217         23       15   
2        16           Asal Usul Laut Tador            93          8        8   
3        23               Rawang Tengkuluk           170         11        7   
4        25               Kisah Doyan Nada           164         15       15   
5        26           Asal Mula Pulau Nusa           139         10       10   
6        35                  Banta Seudang           191         19       15   
7        87                  Tanjung Siman            38          7        2   
8       109                        Ine Ude           155         11        9   
9       111                 Mentiko Betuah           173         14       13   

   precision  recall  f1_score  
0      0.732   0.469     0.571  
1      0.652   0.214     0.323  
2      1.000   0.421

---
### **PSEUDOLABELLING ROUND**
---


---
#### **ROUND 1**
---

In [ ]:
import pandas as pd
from transformers import TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification
from collections import defaultdict

# Load and group the unlabeled data
df_unlabeled = pd.read_csv("/content/cerita_rakyat_tokenized_clean.csv")

# Group into sentences
grouped = df_unlabeled.groupby(['story_id', 'sentence_id'])
sentence_map = []
for (story_id, sentence_id), group in grouped:
    words = group['word'].astype(str).tolist()
    sentence_map.append((story_id, sentence_id, words))

# Load fine-tuned model
model = AutoModelForTokenClassification.from_pretrained("./ner_model")
tokenizer = AutoTokenizer.from_pretrained("./ner_model")

pseudo_pipe = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# Store sentence-level characters
sentence_character_data = []

# Store confidence scores by story
story_confidence_data = defaultdict(list)

for story_id, sentence_id, tokens in sentence_map:
    sentence_text = " ".join(tokens)
    preds = pseudo_pipe(sentence_text)

    char_names = set()

    for pred in preds:
        if pred["entity_group"] == "PER" and pred["score"] >= 0.90:
            char_name = pred["word"].replace("##", "").strip().title()
            if char_name and len(char_name) > 1 and not char_name.isspace():
                char_names.add(char_name)
                story_confidence_data[story_id].append(pred["score"])

    if char_names:
        sentence_character_data.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "characters": list(char_names)
        })

# Save sentence-level character mentions
df_chars = pd.DataFrame(sentence_character_data)
df_chars.to_csv("pseudo_characters_by_sentence.csv", index=False)

# Build story-level confidence summary
story_conf_stats = []
for story_id, confs in story_confidence_data.items():
    story_conf_stats.append({
        "story_id": story_id,
        "avg_confidence": round(sum(confs) / len(confs), 4),
        "min_confidence": round(min(confs), 4),
        "max_confidence": round(max(confs), 4),
        "mention_count": len(confs)
    })

df_conf = pd.DataFrame(story_conf_stats).sort_values("avg_confidence", ascending=False)
df_conf.to_csv("confidence_per_story.csv", index=False)

print(f"✅ Saved {len(df_chars)} rows of sentence-level character mentions.")
print(f"📊 Saved confidence stats for {len(df_conf)} stories.")


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Saved 8536 rows of sentence-level character mentions.
📊 Saved confidence stats for 222 stories.


In [ ]:
import pandas as pd

# Load the sentence-level character data
df = pd.read_csv("pseudo_characters_by_sentence.csv")

# Count unique story IDs
num_stories = df["story_id"].nunique()

print(f"📖 Number of unique stories with character predictions: {num_stories}")


📖 Number of unique stories with character predictions: 222


In [ ]:
import pandas as pd
import torch
from transformers import TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification
from collections import defaultdict

# === Parameters ===
confidence_threshold = 0.95  # Set your threshold (95%)

# === Step 1: Load Unlabeled Data ===
df_unlabeled = pd.read_csv("/content/cerita_rakyat_tokenized_clean.csv")

# Group into sentences
grouped = df_unlabeled.groupby(['story_id', 'sentence_id'])
sentence_map = []
for (story_id, sentence_id), group in grouped:
    words = group['word'].astype(str).tolist()
    sentence_map.append((story_id, sentence_id, words))

# === Step 2: Load Model ===
model = AutoModelForTokenClassification.from_pretrained("./ner_model")
tokenizer = AutoTokenizer.from_pretrained("./ner_model")

pseudo_pipe = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# === Step 3: Pseudo-Labeling ===
high_conf_pseudo_labels = []  # Will be used to augment training data
leftover_sentences = []       # Sentences that need reprocessing later

for story_id, sentence_id, tokens in sentence_map:
    sentence_text = " ".join(tokens)
    preds = pseudo_pipe(sentence_text)

    char_names = []
    char_confidences = []
    low_conf = False

    for pred in preds:
        if pred["entity_group"] == "PER":
            if pred["score"] >= confidence_threshold:
                char_name = pred["word"].replace("##", "").strip().title()
                if char_name and len(char_name) > 1 and not char_name.isspace():
                    char_names.append(char_name)
                    char_confidences.append(round(pred["score"], 4))
            else:
                low_conf = True

    if char_names:
        high_conf_pseudo_labels.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "characters": list(set(char_names)),
            "confidences": char_confidences
        })

    # If any low confidence detected, add sentence to leftovers
    if low_conf:
        leftover_sentences.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "tokens": tokens
        })

# === Step 4: Save Results ===

# Save high confidence pseudo labels
df_high_conf = pd.DataFrame(high_conf_pseudo_labels)
df_high_conf.to_csv("pseudo_characters_high_confidence.csv", index=False)

# Save leftover sentences for next round
df_leftover = pd.DataFrame(leftover_sentences)
df_leftover.to_csv("leftover_sentences_for_next_round.csv", index=False)

print(f"✅ High-confidence pseudo-labeled sentences: {len(df_high_conf)}")
print(f"🔄 Sentences saved for next pseudo-labeling round: {len(df_leftover)}")


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ High-confidence pseudo-labeled sentences: 7880
🔄 Sentences saved for next pseudo-labeling round: 3797



---
#### **ROUND 2**
---

In [ ]:
import pandas as pd

# Load original sentence-tokenized data
df_tokens = pd.read_csv("/content/cerita_rakyat_tokenized_clean.csv")

# Load sentence-level pseudo labels
df_pseudo = pd.read_csv("pseudo_characters_high_confidence.csv")
df_pseudo['characters'] = df_pseudo['characters'].apply(eval)  # Convert from string to list

# Merge pseudo labels with the token-level data
joined_ids = set(zip(df_pseudo['story_id'], df_pseudo['sentence_id']))
df_matched = df_tokens[df_tokens.apply(lambda row: (row['story_id'], row['sentence_id']) in joined_ids, axis=1)].copy()

# Merge character labels into each sentence row
df_merged = df_matched.merge(df_pseudo[['story_id', 'sentence_id', 'characters']], on=['story_id', 'sentence_id'], how='left')

# Function to assign B-PER / I-PER / O
def label_token(word, char_list):
    word = str(word).lower()  # <- force word to be string first
    for char in char_list:
        char_tokens = char.lower().split()
        if word == char_tokens[0]:
            return 'B-PER' if len(char_tokens) == 1 else 'B-PER'
        elif word in char_tokens:
            return 'I-PER'
    return 'O'

# Apply labeling
df_merged['TYPE2'] = df_merged.apply(lambda row: label_token(row['word'], row['characters']), axis=1)

# Drop extras
df_pseudo_labeled = df_merged[['story_id', 'sentence_id', 'word', 'TYPE2']]

# Save as pseudo training data
df_pseudo_labeled.to_csv("pseudo_word_level_labeled_v1.csv", index=False)


In [ ]:
# Load original manually labeled dataset
df_manual = pd.read_csv("/content/TA_MASTER SHEET - even_semi_annotated.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]

# Load pseudo-labeled data
df_pseudo = pd.read_csv("pseudo_word_level_labeled_v1.csv")

# Combine both
df_train = pd.concat([df_manual, df_pseudo], ignore_index=True)

# Save final training set
df_train.to_csv("training_data_for_model_v2.csv", index=False)


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# === Step 1: Load combined dataset ===
df = pd.read_csv("training_data_for_model_v2.csv")
df = df[['story_id', 'sentence_id', 'word', 'TYPE2']]

# Force all words to be string and no NaN
df['word'] = df['word'].fillna('').astype(str)

# Build label mappings
labels = sorted(df['TYPE2'].unique())
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Assign token-level fields
df['ner_tags'] = df['TYPE2'].map(label2id)
df = df[['story_id', 'sentence_id', 'word', 'ner_tags']]

# === Step 2: Group by sentences ===
grouped = df.groupby(['story_id', 'sentence_id'])
sentences = []
tags = []

for _, group in grouped:
    sentences.append(group['word'].tolist())
    tags.append(group['ner_tags'].tolist())

dataset = Dataset.from_dict({
    'tokens': sentences,
    'ner_tags': tags
})


# === Step 3: Train/test split ===
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# === Step 4: Tokenizer and Model ===
model_checkpoint = "cahya/bert-base-indonesian-1.5G"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# === Step 5: Tokenize and Align Labels ===
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# === Step 6: Metric computation ===
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    true_labels = [[id2label[label] for label in sent if label != -100] for sent in labels]
    true_preds = [[id2label[pred] for (pred, label) in zip(sent_pred, sent_label) if label != -100] for sent_pred, sent_label in zip(preds, labels)]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds)
    }

# === Step 7: Training Arguments ===
training_args = TrainingArguments(
    output_dir="./ner_model_v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs_v2",
    logging_steps=100,
    report_to="tensorboard"
)

# === Step 8: Trainer Setup ===
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === Step 9: Train ===
trainer.train()

# Save final model
trainer.save_model("./ner_model_v2")
tokenizer.save_pretrained("./ner_model_v2")

print("✅ Training complete! Model saved at ./ner_model_v2")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6794 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1699 [00:00<?, ? examples/s]

/tmp/ipython-input-32-437296244.py:112: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.080300,0.063274,0.892196,0.915550,0.903722
2,0.045900,0.067779,0.864159,0.937675,0.899417
3,0.029300,0.068938,0.907344,0.912434,0.909882
4,0.019800,0.086812,0.881490,0.936429,0.908129
5,0.011800,0.093343,0.906059,0.922717,0.914312
6,0.008300,0.111261,0.920770,0.909006,0.914850
7,0.003700,0.110833,0.903128,0.926768,0.914795
8,0.004200,0.119210,0.896645,0.932689,0.914312
9,0.002200,0.122080,0.905793,0.925834,0.915703
10,0.001200,0.126250,0.900933,0.932378,0.916386


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.p

✅ Training complete! Model saved at ./ner_model_v2


In [ ]:
import pandas as pd
import torch
from transformers import TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification

# === Step 1: Load model v2 ===
model = AutoModelForTokenClassification.from_pretrained("./ner_model_v2")
tokenizer = AutoTokenizer.from_pretrained("./ner_model_v2")

pipeline = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# === Step 2: Load leftovers ===
df_leftover = pd.read_csv("leftover_sentences_for_next_round.csv")
df_leftover['tokens'] = df_leftover['tokens'].apply(eval)  # Convert from string to list

# === Step 3: Pseudo-labeling round 2 ===
high_conf_pseudo_labels = []
still_low_conf = []

confidence_threshold = 0.90

for _, row in df_leftover.iterrows():
    story_id = row['story_id']
    sentence_id = row['sentence_id']
    tokens = row['tokens']
    sentence = " ".join(tokens)

    preds = pipeline(sentence)

    char_names = []
    confidences = []
    low_conf = False

    for pred in preds:
        if pred["entity_group"] == "PER":
            score = pred["score"]
            word = pred["word"].replace("##", "").strip().title()

            if score >= confidence_threshold:
                if word and len(word) > 1 and not word.isspace():
                    char_names.append(word)
                    confidences.append(round(score, 4))
            else:
                low_conf = True

    if char_names:
        high_conf_pseudo_labels.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "characters": list(set(char_names)),
            "confidences": confidences
        })

    if low_conf:
        still_low_conf.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "tokens": tokens
        })

# === Step 4: Save outputs ===
pd.DataFrame(high_conf_pseudo_labels).to_csv("pseudo_characters_round2.csv", index=False)
pd.DataFrame(still_low_conf).to_csv("leftover_sentences_for_next_round2.csv", index=False)

print(f"✅ Round 2 pseudo-labeled sentences: {len(high_conf_pseudo_labels)}")
print(f"🔄 Still left for round 3: {len(still_low_conf)}")


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Round 2 pseudo-labeled sentences: 3195
🔄 Still left for round 3: 314


In [ ]:
import pandas as pd

# Load both rounds
df_r1 = pd.read_csv("pseudo_characters_high_confidence.csv")
df_r2 = pd.read_csv("pseudo_characters_round2.csv")

# Convert character/confidences to lists if needed
df_r1['characters'] = df_r1['characters'].apply(eval)
df_r2['characters'] = df_r2['characters'].apply(eval)
df_r1['confidences'] = df_r1['confidences'].apply(eval)
df_r2['confidences'] = df_r2['confidences'].apply(eval)

# Concatenate both
df_combined = pd.concat([df_r1, df_r2], ignore_index=True)

# Drop any duplicates (just in case same sentence appeared in both rounds)
df_combined = df_combined.drop_duplicates(subset=['story_id', 'sentence_id'])

# Save merged result
df_combined.to_csv("pseudo_characters_combined_round1_2.csv", index=False)

# Count total
print(f"✅ Total pseudo-labeled sentences after round 2: {len(df_combined)}")


✅ Total pseudo-labeled sentences after round 2: 9209


---
#### **ROUND 3**
---

In [ ]:
import pandas as pd

# Load sentence-tokenized full dataset
df_tokens = pd.read_csv("/content/cerita_rakyat_tokenized_clean.csv")

# Load pseudo-labeled sentence-level characters
df_chars = pd.read_csv("pseudo_characters_combined_round1_2.csv")
df_chars['characters'] = df_chars['characters'].apply(eval)

# Filter token rows that match labeled sentences
matched_ids = set(zip(df_chars['story_id'], df_chars['sentence_id']))
df_matched = df_tokens[df_tokens.apply(lambda row: (row['story_id'], row['sentence_id']) in matched_ids, axis=1)].copy()

# Merge character names to each token row
df_merge = df_matched.merge(df_chars[['story_id', 'sentence_id', 'characters']], on=['story_id', 'sentence_id'], how='left')

# Clean up possible float words
df_merge['word'] = df_merge['word'].fillna('').astype(str)

# Labeling function: B-PER / I-PER / O
def label_token(word, char_list):
    word = word.lower()
    for char in char_list:
        tokens = char.lower().split()
        if word == tokens[0]:
            return 'B-PER' if len(tokens) == 1 else 'B-PER'
        elif word in tokens:
            return 'I-PER'
    return 'O'

df_merge['TYPE2'] = df_merge.apply(lambda row: label_token(row['word'], row['characters']), axis=1)

# Save word-level pseudo-labeled data
df_pseudo = df_merge[['story_id', 'sentence_id', 'word', 'TYPE2']]
df_pseudo.to_csv("pseudo_word_level_labeled_combined_round1_2.csv", index=False)


In [ ]:
# Load original manually labeled data
df_manual = pd.read_csv("/content/TA_MASTER SHEET - even_semi_annotated.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]

# Load new pseudo-labeled tokens
df_pseudo = pd.read_csv("pseudo_word_level_labeled_combined_round1_2.csv")

# Combine for training model v3
df_train = pd.concat([df_manual, df_pseudo], ignore_index=True)
df_train.to_csv("training_data_for_model_v3.csv", index=False)

print("✅ Training dataset for ner_model_v3 is ready!")


✅ Training dataset for ner_model_v3 is ready!


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# === Step 1: Load & preprocess ===
df = pd.read_csv("training_data_for_model_v3.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]
df['word'] = df['word'].fillna('').astype(str)

# Label mapping
labels = sorted(df['TYPE2'].unique())
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Map TYPE2 to numeric
df['ner_tags'] = df['TYPE2'].map(label2id)

# Group into sentences
grouped = df.groupby(['story_id', 'sentence_id'])
sentences = []
tags = []

for _, group in grouped:
    sentences.append(group['word'].tolist())
    tags.append(group['ner_tags'].tolist())

# HuggingFace Dataset
dataset = Dataset.from_dict({
    'tokens': sentences,
    'ner_tags': tags
})

# 80/20 split
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# === Step 2: Tokenizer and Model ===
checkpoint = "cahya/bert-base-indonesian-1.5G"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# === Step 3: Tokenize and align labels ===
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# === Step 4: Metrics ===
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    true_labels = [[id2label[l] for l in sent if l != -100] for sent in labels]
    true_preds = [[id2label[p] for p, l in zip(sent_pred, sent_label) if l != -100]
                  for sent_pred, sent_label in zip(preds, labels)]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds)
    }

# === Step 5: TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./ner_model_v3",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs_v3",
    logging_steps=100,
    report_to="tensorboard"
)

# === Step 6: Trainer ===
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === Step 7: Train ===
trainer.train()

# Save final model
trainer.save_model("./ner_model_v3")
tokenizer.save_pretrained("./ner_model_v3")

print("✅ Training complete! Model saved to ./ner_model_v3 🎉")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7832 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1958 [00:00<?, ? examples/s]

/tmp/ipython-input-40-3866255030.py:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.087100,0.084884,0.826460,0.886316,0.855342
2,0.060400,0.089488,0.822967,0.918967,0.868321
3,0.030500,0.099464,0.835268,0.897002,0.865035
4,0.018600,0.115582,0.836216,0.918373,0.875371
5,0.013000,0.119755,0.849519,0.916592,0.881782
6,0.009300,0.131134,0.842814,0.910359,0.875285
7,0.005200,0.158243,0.841707,0.924904,0.881346
8,0.003600,0.153806,0.854947,0.907985,0.880668
9,0.003000,0.157254,0.856465,0.910359,0.882590
10,0.001700,0.168796,0.847360,0.914515,0.879657


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.p

✅ Training complete! Model saved to ./ner_model_v3 🎉


In [ ]:
import pandas as pd
import torch
from transformers import TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification

# === Step 1: Load ner_model_v3 ===
model = AutoModelForTokenClassification.from_pretrained("./ner_model_v3")
tokenizer = AutoTokenizer.from_pretrained("./ner_model_v3")

pipeline = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # merge subwords nicely
    device=0 if torch.cuda.is_available() else -1
)

# === Step 2: Load leftover sentences ===
df_leftover = pd.read_csv("leftover_sentences_for_next_round2.csv")
df_leftover['tokens'] = df_leftover['tokens'].apply(eval)  # convert string to list

# === Step 3: Pseudo-labeling leftover round 3 ===
confidence_threshold = 0.90

high_conf_pseudo_labels = []
still_low_conf = []

for _, row in df_leftover.iterrows():
    story_id = row['story_id']
    sentence_id = row['sentence_id']
    tokens = row['tokens']
    sentence = " ".join(tokens)

    preds = pipeline(sentence)

    char_names = []
    confidences = []
    low_conf = False

    for pred in preds:
        if pred["entity_group"] == "PER":
            score = pred["score"]
            word = pred["word"].replace("##", "").strip().title()

            if score >= confidence_threshold:
                if word and len(word) > 1 and not word.isspace():
                    char_names.append(word)
                    confidences.append(round(score, 4))
            else:
                low_conf = True

    if char_names:
        high_conf_pseudo_labels.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "characters": list(set(char_names)),
            "confidences": confidences
        })

    if low_conf:
        still_low_conf.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "tokens": tokens
        })

# === Step 4: Save results ===
pd.DataFrame(high_conf_pseudo_labels).to_csv("pseudo_characters_round3.csv", index=False)
pd.DataFrame(still_low_conf).to_csv("leftover_sentences_for_next_round3.csv", index=False)

print(f"✅ Round 3 pseudo-labeled sentences: {len(high_conf_pseudo_labels)}")
print(f"🔄 Still left for round 4: {len(still_low_conf)}")


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Round 3 pseudo-labeled sentences: 197
🔄 Still left for round 4: 56


In [ ]:
import pandas as pd

# Load all rounds
df_r1 = pd.read_csv("pseudo_characters_high_confidence.csv")
df_r2 = pd.read_csv("pseudo_characters_round2.csv")
df_r3 = pd.read_csv("pseudo_characters_round3.csv")

# Ensure lists are evaluated properly
for df in [df_r1, df_r2, df_r3]:
    df['characters'] = df['characters'].apply(eval)
    df['confidences'] = df['confidences'].apply(eval)

# Merge
df_all = pd.concat([df_r1, df_r2, df_r3], ignore_index=True)

# Drop duplicate sentences just in case
df_all = df_all.drop_duplicates(subset=['story_id', 'sentence_id'])

# Save combined result
df_all.to_csv("pseudo_characters_combined_round1_2_3.csv", index=False)

print(f"✅ Total pseudo-labeled sentences after round 3: {len(df_all)}")


✅ Total pseudo-labeled sentences after round 3: 9282


## pseudo labelling round 4

In [ ]:
import pandas as pd

# Load tokenized dataset and pseudo characters
df_tokens = pd.read_csv("/content/cerita_rakyat_tokenized_clean.csv")
df_chars = pd.read_csv("pseudo_characters_combined_round1_2_3.csv")

df_chars['characters'] = df_chars['characters'].apply(eval)

# Match sentences
matched_ids = set(zip(df_chars['story_id'], df_chars['sentence_id']))
df_matched = df_tokens[df_tokens.apply(lambda row: (row['story_id'], row['sentence_id']) in matched_ids, axis=1)].copy()

# Merge character info into matched tokens
df_merge = df_matched.merge(df_chars[['story_id', 'sentence_id', 'characters']], on=['story_id', 'sentence_id'], how='left')

df_merge['word'] = df_merge['word'].fillna('').astype(str)

# Label each token with B-PER/I-PER/O
def label_token(word, char_list):
    word = word.lower()
    for char in char_list:
        tokens = char.lower().split()
        if word == tokens[0]:
            return 'B-PER' if len(tokens) == 1 else 'B-PER'
        elif word in tokens:
            return 'I-PER'
    return 'O'

df_merge['TYPE2'] = df_merge.apply(lambda row: label_token(row['word'], row['characters']), axis=1)

# Save token-level pseudo labels
df_pseudo_tokens = df_merge[['story_id', 'sentence_id', 'word', 'TYPE2']]
df_pseudo_tokens.to_csv("pseudo_word_level_labeled_round1_2_3.csv", index=False)

print("✅ Token-level pseudo labels saved for ner_model_v4")


✅ Token-level pseudo labels saved for ner_model_v4


In [ ]:
# Load manual labels
df_manual = pd.read_csv("/content/TA_MASTER SHEET - even_semi_annotated.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]

# Load pseudo labels
df_pseudo = pd.read_csv("pseudo_word_level_labeled_round1_2_3.csv")

# Merge them
df_final = pd.concat([df_manual, df_pseudo], ignore_index=True)
df_final.to_csv("training_data_for_model_v4.csv", index=False)

print("✅ Training data ready for ner_model_v4 🎉")


✅ Training data ready for ner_model_v4 🎉


---
#### **ROUND 4**
---

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# === Step 1: Load training dataset ===
df = pd.read_csv("training_data_for_model_v4.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]
df['word'] = df['word'].fillna('').astype(str)

# Build label mappings
labels = sorted(df['TYPE2'].unique())
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Map TYPE2 to numeric
df['ner_tags'] = df['TYPE2'].map(label2id)

# Group into sentences
grouped = df.groupby(['story_id', 'sentence_id'])
sentences = []
tags = []

for _, group in grouped:
    sentences.append(group['word'].tolist())
    tags.append(group['ner_tags'].tolist())

# HuggingFace Dataset
dataset = Dataset.from_dict({
    'tokens': sentences,
    'ner_tags': tags
})

# Train/test split (80/20)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# === Step 2: Tokenizer and Model ===
checkpoint = "cahya/bert-base-indonesian-1.5G"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# === Step 3: Tokenize and align labels ===
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# === Step 4: Metric Computation ===
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    true_labels = [[id2label[label] for label in sent if label != -100] for sent in labels]
    true_preds = [[id2label[pred] for (pred, label) in zip(sent_pred, sent_label) if label != -100]
                  for sent_pred, sent_label in zip(preds, labels)]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds)
    }

# === Step 5: TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./ner_model_v4",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs_v4",
    logging_steps=100,
    report_to="tensorboard"
)

# 💬 Add Early Stopping
from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

# === Step 6: Trainer ===
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === Step 7: Start Training ===
trainer.train()

# Save final model
trainer.save_model("./ner_model_v4")
tokenizer.save_pretrained("./ner_model_v4")

print("✅ Training complete! Model saved at ./ner_model_v4 🎉")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7887 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1972 [00:00<?, ? examples/s]

/tmp/ipython-input-47-1621527156.py:117: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.087100,0.078241,0.852131,0.886042,0.868756
2,0.055600,0.078875,0.841305,0.917395,0.877704
3,0.038300,0.094151,0.838843,0.917998,0.876637
4,0.020600,0.101108,0.864716,0.899910,0.881962
5,0.012200,0.118371,0.861969,0.892373,0.876907
6,0.008300,0.116424,0.865100,0.899005,0.881727
7,0.005900,0.129963,0.871533,0.899910,0.885494
8,0.002900,0.145421,0.856820,0.914682,0.884806
9,0.003000,0.146048,0.860458,0.905336,0.882327
10,0.001200,0.145495,0.866301,0.904432,0.884956


✅ Training complete! Model saved at ./ner_model_v4 🎉


In [ ]:
import pandas as pd
import torch
from transformers import TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification

# === Step 1: Load ner_model_v4 ===
model = AutoModelForTokenClassification.from_pretrained("./ner_model_v4")
tokenizer = AutoTokenizer.from_pretrained("./ner_model_v4")

pipeline = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# === Step 2: Load leftover sentences ===
df_leftover = pd.read_csv("leftover_sentences_for_next_round3.csv")
df_leftover['tokens'] = df_leftover['tokens'].apply(eval)  # Convert from string to list

# === Step 3: Pseudo-labeling leftover round 4 ===
confidence_threshold = 0.90

high_conf_pseudo_labels = []
still_low_conf = []

for _, row in df_leftover.iterrows():
    story_id = row['story_id']
    sentence_id = row['sentence_id']
    tokens = row['tokens']
    sentence = " ".join(tokens)

    preds = pipeline(sentence)

    char_names = []
    confidences = []
    low_conf = False

    for pred in preds:
        if pred["entity_group"] == "PER":
            score = pred["score"]
            word = pred["word"].replace("##", "").strip().title()

            if score >= confidence_threshold:
                if word and len(word) > 1 and not word.isspace():
                    char_names.append(word)
                    confidences.append(round(score, 4))
            else:
                low_conf = True

    if char_names:
        high_conf_pseudo_labels.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "characters": list(set(char_names)),
            "confidences": confidences
        })

    if low_conf:
        still_low_conf.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "tokens": tokens
        })

# === Step 4: Save results ===
pd.DataFrame(high_conf_pseudo_labels).to_csv("pseudo_characters_round4.csv", index=False)
pd.DataFrame(still_low_conf).to_csv("leftover_sentences_for_next_round4.csv", index=False)

print(f"✅ Round 4 pseudo-labeled sentences: {len(high_conf_pseudo_labels)}")
print(f"🔄 Still left for round 5: {len(still_low_conf)}")


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Round 4 pseudo-labeled sentences: 33
🔄 Still left for round 5: 16


In [ ]:
import pandas as pd

# Load all pseudo-labeled rounds
df_r1 = pd.read_csv("pseudo_characters_high_confidence.csv")
df_r2 = pd.read_csv("pseudo_characters_round2.csv")
df_r3 = pd.read_csv("pseudo_characters_round3.csv")
df_r4 = pd.read_csv("pseudo_characters_round4.csv")

# Ensure 'characters' and 'confidences' columns are real lists
for df in [df_r1, df_r2, df_r3, df_r4]:
    df['characters'] = df['characters'].apply(eval)
    df['confidences'] = df['confidences'].apply(eval)

# Merge everything
df_combined = pd.concat([df_r1, df_r2, df_r3, df_r4], ignore_index=True)

# Drop duplicates (in case same sentence was predicted in multiple rounds)
df_combined = df_combined.drop_duplicates(subset=['story_id', 'sentence_id'])

# Save merged output
df_combined.to_csv("pseudo_characters_combined_round1_2_3_4.csv", index=False)

# Show summary
print(f"✅ Total final pseudo-labeled sentences after round 4: {len(df_combined)}")


✅ Total final pseudo-labeled sentences after round 4: 9297


---
#### **ROUND 5**
---

In [ ]:
import pandas as pd

# Load tokenized dataset and pseudo characters
df_tokens = pd.read_csv("/content/cerita_rakyat_tokenized_clean.csv")
df_chars = pd.read_csv("pseudo_characters_combined_round1_2_3_4.csv")

df_chars['characters'] = df_chars['characters'].apply(eval)

# Match sentences
matched_ids = set(zip(df_chars['story_id'], df_chars['sentence_id']))
df_matched = df_tokens[df_tokens.apply(lambda row: (row['story_id'], row['sentence_id']) in matched_ids, axis=1)].copy()

# Merge character info into matched tokens
df_merge = df_matched.merge(df_chars[['story_id', 'sentence_id', 'characters']], on=['story_id', 'sentence_id'], how='left')

df_merge['word'] = df_merge['word'].fillna('').astype(str)

# Label each token with B-PER/I-PER/O
def label_token(word, char_list):
    word = word.lower()
    for char in char_list:
        tokens = char.lower().split()
        if word == tokens[0]:
            return 'B-PER' if len(tokens) == 1 else 'B-PER'
        elif word in tokens:
            return 'I-PER'
    return 'O'

df_merge['TYPE2'] = df_merge.apply(lambda row: label_token(row['word'], row['characters']), axis=1)

# Save token-level pseudo labels
df_pseudo_tokens = df_merge[['story_id', 'sentence_id', 'word', 'TYPE2']]
df_pseudo_tokens.to_csv("pseudo_word_level_labeled_round1_2_3_4.csv", index=False)

print("✅ Token-level pseudo labels saved for ner_model_v4")


✅ Token-level pseudo labels saved for ner_model_v4


In [ ]:
# Load manual labels
df_manual = pd.read_csv("/content/TA_MASTER SHEET - even_semi_annotated.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]

# Load pseudo labels
df_pseudo = pd.read_csv("pseudo_word_level_labeled_round1_2_3_4.csv")

# Merge them
df_final = pd.concat([df_manual, df_pseudo], ignore_index=True)
df_final.to_csv("training_data_for_model_v5.csv", index=False)

print("✅ Training data ready for ner_model_v5 🎉")


✅ Training data ready for ner_model_v5 🎉


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# === Step 1: Load training dataset ===
df = pd.read_csv("training_data_for_model_v5.csv")[['story_id', 'sentence_id', 'word', 'TYPE2']]
df['word'] = df['word'].fillna('').astype(str)

# Build label mappings
labels = sorted(df['TYPE2'].unique())
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Map TYPE2 to numeric
df['ner_tags'] = df['TYPE2'].map(label2id)

# Group into sentences
grouped = df.groupby(['story_id', 'sentence_id'])
sentences = []
tags = []

for _, group in grouped:
    sentences.append(group['word'].tolist())
    tags.append(group['ner_tags'].tolist())

# HuggingFace Dataset
dataset = Dataset.from_dict({
    'tokens': sentences,
    'ner_tags': tags
})

# Train/test split (80/20)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# === Step 2: Tokenizer and Model ===
checkpoint = "cahya/bert-base-indonesian-1.5G"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# === Step 3: Tokenize and align labels ===
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# === Step 4: Metric Computation ===
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    true_labels = [[id2label[label] for label in sent if label != -100] for sent in labels]
    true_preds = [[id2label[pred] for (pred, label) in zip(sent_pred, sent_label) if label != -100]
                  for sent_pred, sent_label in zip(preds, labels)]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds)
    }

# === Step 5: TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./ner_model_v5",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs_v4",
    logging_steps=100,
    report_to="tensorboard"
)

# 💬 Add Early Stopping
from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

# === Step 6: Trainer ===
data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# === Step 7: Start Training ===
trainer.train()

# Save final model
trainer.save_model("./ner_model_v5")
tokenizer.save_pretrained("./ner_model_v5")

print("✅ Training complete! Model saved at ./ner_model_v5 🎉")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7898 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1975 [00:00<?, ? examples/s]

/tmp/ipython-input-53-3710295240.py:117: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.091700,0.078219,0.851590,0.872661,0.861997
2,0.051600,0.074832,0.854048,0.904043,0.878335
3,0.034100,0.081018,0.852111,0.919734,0.884632
4,0.023000,0.097465,0.857389,0.899819,0.878092
5,0.012200,0.109798,0.866358,0.903742,0.884655
6,0.007700,0.122765,0.862296,0.908871,0.884971
7,0.006900,0.126634,0.865840,0.905552,0.885251
8,0.004600,0.134135,0.856374,0.912191,0.883402
9,0.002700,0.142010,0.867442,0.900422,0.883625
10,0.002700,0.147557,0.863832,0.907363,0.885063


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.p

✅ Training complete! Model saved at ./ner_model_v5 🎉


In [ ]:
import pandas as pd
import torch
from transformers import TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification

# === Step 1: Load ner_model_v4 ===
model = AutoModelForTokenClassification.from_pretrained("./ner_model_v5")
tokenizer = AutoTokenizer.from_pretrained("./ner_model_v5")

pipeline = TokenClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# === Step 2: Load leftover sentences ===
df_leftover = pd.read_csv("leftover_sentences_for_next_round4.csv")
df_leftover['tokens'] = df_leftover['tokens'].apply(eval)  # Convert from string to list

# === Step 3: Pseudo-labeling leftover round 4 ===
confidence_threshold = 0.90

high_conf_pseudo_labels = []
still_low_conf = []

for _, row in df_leftover.iterrows():
    story_id = row['story_id']
    sentence_id = row['sentence_id']
    tokens = row['tokens']
    sentence = " ".join(tokens)

    preds = pipeline(sentence)

    char_names = []
    confidences = []
    low_conf = False

    for pred in preds:
        if pred["entity_group"] == "PER":
            score = pred["score"]
            word = pred["word"].replace("##", "").strip().title()

            if score >= confidence_threshold:
                if word and len(word) > 1 and not word.isspace():
                    char_names.append(word)
                    confidences.append(round(score, 4))
            else:
                low_conf = True

    if char_names:
        high_conf_pseudo_labels.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "characters": list(set(char_names)),
            "confidences": confidences
        })

    if low_conf:
        still_low_conf.append({
            "story_id": story_id,
            "sentence_id": sentence_id,
            "tokens": tokens
        })

# === Step 4: Save results ===
pd.DataFrame(high_conf_pseudo_labels).to_csv("pseudo_characters_round5.csv", index=False)
pd.DataFrame(still_low_conf).to_csv("leftover_sentences_for_next_round5.csv", index=False)

print(f"✅ Round 4 pseudo-labeled sentences: {len(high_conf_pseudo_labels)}")
print(f"🔄 Still left for round 5: {len(still_low_conf)}")


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Round 4 pseudo-labeled sentences: 9
🔄 Still left for round 5: 7


In [ ]:
import pandas as pd

# Load all pseudo-labeled rounds
df_r1 = pd.read_csv("pseudo_characters_high_confidence.csv")
df_r2 = pd.read_csv("pseudo_characters_round2.csv")
df_r3 = pd.read_csv("pseudo_characters_round3.csv")
df_r4 = pd.read_csv("pseudo_characters_round4.csv")
df_r5 = pd.read_csv("pseudo_characters_round5.csv")

# Ensure 'characters' and 'confidences' columns are real lists
for df in [df_r1, df_r2, df_r3, df_r4, df_r5]:
    df['characters'] = df['characters'].apply(eval)
    df['confidences'] = df['confidences'].apply(eval)

# Merge everything
df_combined = pd.concat([df_r1, df_r2, df_r3, df_r4,df_r5], ignore_index=True)

# Drop duplicates (in case same sentence was predicted in multiple rounds)
df_combined = df_combined.drop_duplicates(subset=['story_id', 'sentence_id'])

# Save merged output
df_combined.to_csv("pseudo_characters_combined_round1_2_3_4_5.csv", index=False)

# Show summary
print(f"✅ Total final pseudo-labeled sentences after round 4: {len(df_combined)}")


✅ Total final pseudo-labeled sentences after round 4: 9302
